In [7]:
import pandas as pd
import numpy as np

In [8]:
def get_df(fname):
    df = pd.read_csv('data/'+fname, index_col=0)    
    df = df.T
    return df

fname = 'Test_relia100_Tyler_20200113_203252.csv'
df = get_df(fname)
df.head()

case name,system reliability,CO2 price ($/kgCO2),norm. demand to 1,demand file,mean demand (kW),fixed cost natgas ($/kW/h),var cost natgas ($/kW/h),fixed cost natgas_ccs ($/kW/h),var cost natgas_ccs ($/kW/h),fixed cost solar ($/kW/h),...,dispatch wind (kW),curtailment wind (kW),capacity nuclear (kW),dispatch nuclear (kW),curtailment nuclear (kW),capacity storage (kW),energy storage (kW),dispatch to storage (kW),dispatch from storage (kW),dispatch unmet demand (kW)
CONUS_BAs_MEM1,1,0,TRUE,demand_series_Tyler_mean_CONUS_from_BAs_for_ME...,1,0.0118,0.039,0.0273,0.0566,0.0195,...,0,0,0,0,0,0,0,0,0,0
CONUS_BAs_MEM2,1,0,TRUE,demand_series_Tyler_mean_CONUS_from_BAs_for_ME...,1,-1,-1,-1,-1,0.0195,...,0.445994021,0.004490914,0.920856164,0.379928582,0.540927583,1.181656019,0.245564506,0.006726295,0.006053386,0
CONUS_BAs_MEM3,1,0,TRUE,demand_series_Tyler_mean_CONUS_from_BAs_for_ME...,1,-1,-1,-1,-1,0.0195,...,0.575613983,1.541731699,0,0,0,4.868800873,0.785343449,0.004576982,0.004118388,0
CONUS_BAs_impute1,1,0,TRUE,demand_series_Tyler_mean_CONUS_from_BAs_mean_i...,1,0.0118,0.039,0.0273,0.0566,0.0195,...,0,0,0,0,0,0,0,0,0,0
CONUS_BAs_impute2,1,0,TRUE,demand_series_Tyler_mean_CONUS_from_BAs_mean_i...,1,-1,-1,-1,-1,0.0195,...,0.149469752,0.064945997,0.298396997,0.020691121,0.277705876,32.36317105,6.895125351,0.370662582,0.333588463,0


In [9]:
def demand_info(df, idx):
    dem = pd.read_csv('~/Downloads/Tyler_Data_3/'+df.loc[idx, 'demand file'], header=5)
    max_ = np.max(dem['demand'])
    min_ = np.min(dem['demand'])
    mean_ = np.mean(dem['demand'])
    std_ = np.std(dem['demand'])
    #idxs = dem['demand'] == max_
    #print(dem.loc[idxs, 'year'],dem.loc[idxs, 'month'],dem.loc[idxs, 'day'],dem.loc[idxs, 'hour'],dem.loc[idxs, 'demand'])
    return dem, max_, min_, mean_, std_


def print_case(fname, tgt_case, scenario, print_cap=True):
    df = get_df(fname)
    if scenario == '1':
        test = 'EIA Base'
    elif scenario == '2':
        test = 'No NatGas'
    elif scenario == '3':
        test = 'No NatGas, no nuclear'
    
    order = ['Scenario 1','Scenario 2','Scenario 3']
    save_map = {}
    save_map_dem = {}
    
    for idx in df.index:
        #if idx[-4:] == 'MEM1':
        if tgt_case in idx and idx[-1] == scenario:
            if 'MEM' in idx:
                case_code = 'Scenario 3'
            elif 'impute' in idx:
                case_code = 'N/A'
                continue
            elif 'REGIONS' in idx:
                case_code = 'Scenario 2'
            else:
                case_code = 'Scenario 1'
            
            dem, max_, min_, mean_, std_ = demand_info(df, idx)
            
            val = 0
            ng = round(float(df.loc[idx, 'capacity natgas (kW)'])*mean_/1000)
            wind = round(float(df.loc[idx, 'capacity wind (kW)'])*mean_/1000)
            solar = round(float(df.loc[idx, 'capacity solar (kW)'])*mean_/1000)
            nuke = round(float(df.loc[idx, 'capacity nuclear (kW)'])*mean_/1000)
            storage = round(float(df.loc[idx, 'capacity storage (kW)'])*mean_/1000)
            save_map[case_code] = f'{tgt_case}:{test}:{case_code}:{ng}:{wind}:{solar}:{nuke}:{storage}'
            save_map_dem[case_code] = f'{tgt_case}:{case_code}:{mean_}:{max_}:{round(max_/mean_*100)}:{min_}:{round(min_/mean_*100)}'#:{round(std_/mean_*100,2)}'
    
    if print_cap:
        for s in order:
            print(save_map[s])
    else:
        for s in order:
            print(save_map_dem[s])
    return



print_case(fname, 'CONUS', '1')

CONUS:EIA Base:Scenario 1:3769:0:0:0:0
CONUS:EIA Base:Scenario 2:708:0:0:0:0
CONUS:EIA Base:Scenario 3:717:0:0:0:0


In [10]:
print("Region:Test:Case:NatGas:Wind:Solar:Nuclear:Storage")
tests = ['CONUS','EASTERN','WESTERN','TEXAS']
for test in tests:
    for i in ['1','2','3']:
        print_case(fname, test, i)

Region:Test:Case:NatGas:Wind:Solar:Nuclear:Storage
CONUS:EIA Base:Scenario 1:3769:0:0:0:0
CONUS:EIA Base:Scenario 2:708:0:0:0:0
CONUS:EIA Base:Scenario 3:717:0:0:0:0
CONUS:No NatGas:Scenario 1:0:203:1951:133:14677
CONUS:No NatGas:Scenario 2:0:473:278:414:566
CONUS:No NatGas:Scenario 3:0:483:290:422:542
CONUS:No NatGas, no nuclear:Scenario 1:0:604:2030:0:14638
CONUS:No NatGas, no nuclear:Scenario 2:0:2216:754:0:2191
CONUS:No NatGas, no nuclear:Scenario 3:0:2269:757:0:2233
EASTERN:EIA Base:Scenario 1:533:0:0:0:0
EASTERN:EIA Base:Scenario 2:534:0:0:0:0
EASTERN:EIA Base:Scenario 3:533:0:0:0:0
EASTERN:No NatGas:Scenario 1:0:76:113:399:461
EASTERN:No NatGas:Scenario 2:0:77:112:400:462
EASTERN:No NatGas:Scenario 3:0:77:106:403:445
EASTERN:No NatGas, no nuclear:Scenario 1:0:1807:512:0:3489
EASTERN:No NatGas, no nuclear:Scenario 2:0:1766:514:0:3755
EASTERN:No NatGas, no nuclear:Scenario 3:0:1818:504:0:3575
WESTERN:EIA Base:Scenario 1:3420:0:0:0:0
WESTERN:EIA Base:Scenario 2:139:0:0:0:0
WESTERN:

In [5]:
print("Region:Test:Mean Demand (MW):Maximum Demand (MW):Max/Mean (%):Minimum Demand (MW):Min/Mean (%)")#:Std/Mean (%)")
tests = ['CONUS','EASTERN','WESTERN','TEXAS']
for test in tests:
    for i in ['1',]:#'2','3']:
        print_case(fname, test, i, False)

Region:Test:Mean Demand (MW):Maximum Demand (MW):Max/Mean (%):Minimum Demand (MW):Min/Mean (%)
CONUS:Scenario 1:455351.9020301095:3768848:828:218042:48
CONUS:Scenario 2:450401.451908455:935346:208:202281:45
CONUS:Scenario 3:458583.6417274939:717466:156:311432:68
EASTERN:Scenario 1:331283.5068810827:533327:161:145984:44
EASTERN:Scenario 2:332490.289651764:533675:161:169064:51
EASTERN:Scenario 3:333669.9602341849:533327:160:218141:65
WESTERN:Scenario 1:83172.61960158151:3419601:4111:37044:45
WESTERN:Scenario 2:83212.72574513382:139310:167:46568:56
WESTERN:Scenario 3:83636.5920772506:139310:167:59270:71
TEXAS:Scenario 1:40895.77554744526:73264:179:0:0
TEXAS:Scenario 2:41209.52976733576:73264:178:0:0
TEXAS:Scenario 3:41277.09447801106:73264.0:177:25146.0:61


In [6]:
### This cell shows why the max demand does not align for Eastern between S1,2,3.
### It is b/c for S1 and S3, I aggregate from the BA level and exclude SEC and OVEC
### while EIA method is based on regions and includes SEC and OVEC

def return_all_regions():
    return ['AEC', 'AECI', 'CPLE', 'CPLW',
    'DUK', 'FMPP', 'FPC',
    'FPL', 'GVL', 'HST', 'ISNE',
    'JEA', 'LGEE', 'MISO', 'NSB',
    'NYIS', 'OVEC', 'PJM', 'SC',
    'SCEG', 'SEC', 'SOCO',
    'SPA', 'SWPP', 'TAL', 'TEC',
    'TVA', 'ERCO',
    'AVA', 'AZPS', 'BANC', 'BPAT',
    'CHPD', 'CISO', 'DOPD',
    'EPE', 'GCPD', 'IID',
    'IPCO', 'LDWP', 'NEVP', 'NWMT',
    'PACE', 'PACW', 'PGE', 'PNM',
    'PSCO', 'PSEI', 'SCL', 'SRP',
    'TEPC', 'TIDC', 'TPWR', 'WACM',
    'WALC', 'WAUW']

regions = return_all_regions()
regions = ['SEC','OVEC']

regions.sort()
print(len(regions))
for region in regions:
    print(region)

    file_path = '~/data4/{}.csv'.format(region)
    df = pd.read_csv(file_path,
                    dtype={'demand (MW)':np.float64},
                    parse_dates=True, na_values=['MISSING', 'EMPTY'])
    print(df.loc[df['time'] == '20160811T21Z'])

2
OVEC
              time  year  month  day  hour  demand (MW)  forecast demand (MW)
9784  20160811T21Z  2016      8   11    21        121.0                 109.0
SEC
              time  year  month  day  hour  demand (MW)  forecast demand (MW)
9784  20160811T21Z  2016      8   11    21        227.0                 278.0


In [19]:
path = '~/Downloads/Updated_SEM_Hour_Format_20190910/'
files = [
'CONUS_from_BAs.csv',
'CONUS_from_REGIONS.csv',
'EASTERN_from_BAs.csv',
'EASTERN_from_REGIONS.csv',
'TEXAS_from_BAs.csv',
'TEXAS_from_REGIONS.csv',
'WESTERN_from_BAs.csv',
'WESTERN_from_REGIONS.csv',
]

def demand_info2(fname):
    dem = pd.read_csv(fname)
    #print(dem.head())
    to_drop = []
    for idx in dem.index:
        if '2015' in dem.loc[idx, 'time'] or '2019' in dem.loc[idx, 'time']:
            to_drop.append(idx)
    dem = dem.drop(to_drop, axis=0)
    #print(dem.head())
    #print(dem.tail())
    max_ = np.max(dem['demand (MW)'])
    min_ = np.min(dem['demand (MW)'])
    mean_ = np.mean(dem['demand (MW)'])
    std_ = np.std(dem['demand (MW)'])
    #idxs = dem['demand'] == max_
    #print(dem.loc[idxs, 'year'],dem.loc[idxs, 'month'],dem.loc[idxs, 'day'],dem.loc[idxs, 'hour'],dem.loc[idxs, 'demand'])
    print(f"{fname}:{mean_/1000}:{max_/1000}:{max_/mean_*100}:{min_/1000}:{min_/mean_*100}")

print("fname:Mean Demand (MW):Maximum Demand (MW):Max/Mean (%):Minimum Demand (MW):Min/Mean (%)")#:Std/Mean (%)")
for fname in files:
    demand_info2(path+fname)

fname:Mean Demand (MW):Maximum Demand (MW):Max/Mean (%):Minimum Demand (MW):Min/Mean (%)
~/Downloads/Updated_SEM_Hour_Format_20190910/CONUS_from_BAs.csv:455.3534371198297:3768.848:827.6753160881926:218.042:47.88412301862577
~/Downloads/Updated_SEM_Hour_Format_20190910/CONUS_from_REGIONS.csv:456.9129822460462:717.848:157.10825209458397:271.855:59.498199999405344
~/Downloads/Updated_SEM_Hour_Format_20190910/EASTERN_from_BAs.csv:331.2849993917275:533.327:160.98736766809301:145.984:44.06598556168896
~/Downloads/Updated_SEM_Hour_Format_20190910/EASTERN_from_REGIONS.csv:332.49068419251824:533.675:160.50825643313132:169.064:50.84774041431754
~/Downloads/Updated_SEM_Hour_Format_20190910/TEXAS_from_BAs.csv:40.89581200577859:73.264:179.1479283738095:0.0:0.0
~/Downloads/Updated_SEM_Hour_Format_20190910/TEXAS_from_REGIONS.csv:41.2095662256691:73.264:177.7839630701195:0.0:0.0
~/Downloads/Updated_SEM_Hour_Format_20190910/WESTERN_from_BAs.csv:83.1726257223236:3419.601:4111.45009587232:37.044:44.53869